# S&P 500 Analysis with Interactive Dashboard

## Overview

This project is the analysis of S&P 500 data gathered from 1985 to 2022. I have chosen to do this to enhance my portfolio of data analysis projects, focussing on finance. I take in a `.csv` file to clean and work with the data to create visuals, calculate technical indicators (SMA, RSI, MACD), and create an interactive dashboard with my findings.

## The Data

I chose the "S&P 500 Stocks" dataset which I found [here](https://www.kaggle.com/datasets/whenamancodes/sp-500-stocks), on kaggle, which can be implemented into your code through importing kaggle into your script or by reading the CSV file, which is the method I chose to do. 

The data contains information on the daily open prices, daily close prices, highs, lows and volume of the S&P 500 on the first of every month between 01/01/1985 and 01/09/2022.

In `load_data.py`, I created a function `read_csv(csv)` which takes in one argument (a `.csv` file), to create a dataframe from the passed in file through using the Pandas library. I then clean the data accordingly, removing the unwanted column, namely "Adj Close", changing the date column to the desired format, casting any float-valued entries to have the datatype of a float, and finally removing any unwanted, non-numeric characters from the floats.

The above python file could be adapted to accomodate for other ways datasets might be presented for example, a function to take in a database, or API.

In `main.py`, after cleaning the data, I began to check for any empty values or duplicates and dealing with them accordingly. Nicely, in this dataset, there were no empty values or duplicates.

We can view the first 5 rows of the data from the csv file to look at what we have to work with:

In [202]:
import pandas as pd 
from load_data import read_csv
from indicators import sma_indicator, rsi_indicator, macd_indicator
from database import rsi_signals

df = read_csv("data/sp 500.csv")

df.head()

,Date,Open,High,Low,Close,Volume
0,2022-09-01,3936.73,4119.28,3610.40,3640.47,8.866705e+10
1,2022-08-01,4112.38,4325.28,3954.53,3955.00,7.808852e+10
2,2022-07-01,3781.00,4140.15,3721.56,4130.29,6.802114e+10
3,2022-06-01,4149.78,4177.51,3636.87,3785.38,8.679906e+10
4,2022-05-01,4130.61,4307.66,3810.32,4132.15,8.912813e+10


## Visualisation

There is one visualisation which is not on the dashboard, and that is a candlestick chart of the S&P 500 between two years of the users choosing. Below, an example between the years 2000 and 2005 can be seen:

<p align="center">
  <img src="images/ohlc_chart_2000_2005.png" alt="Candlestick Chart between 2000 and 2005" width="800">
</p>

## Technical indicators

In my project I calculate three types of technical indicators, namely, Simple Moving Average (SMA), Relative Strength Index (RSI) and Moving Average Convergence Divergence (MACD). I calculate these in a script called `indicators.py` with functions `sma_indicator(df, 20, 50)`, `rsi_indicator(df, 14)`, `macd_indicator(df)`; they take in the data, aswell as the specified periods.

The SMA is calculated by taking the sum of the closing prices in a given day range, and dividing it by the number of days in the range. The example used here is between days 20 and 50. So, both `SMA_20` and `SMA_50` tell you the average price over the last 20 and 50 days respectively. The choice of 20 and 50 days give you a view of both short and medium term trends of the stock.

RSI is used to determine when a stock has been overbought or oversold. In my project I use the RSI to determine exactly this, presenting the days in which the S&P 500 is oversold or overbought.
Normally, the RSI is calculated through the formula 
$$\text{RSI} = 100 - \frac{100}{1-\text{rs}}$$

where $\text{rs}$ is the average gain / average loss.

The MACD is used to potentially signal when to buy or sell. In my scripts I use calculations to see when the S&P 500 is bullish or bearish, creating new columns in the dataframe to show these results.

A snippet of the table after calculating these technical indicators can be seen below. As an example, I chose 5 random rows from the year 2015.

In [203]:
from indicators import sma_indicator, rsi_indicator, macd_indicator

sma_indicator(df, 20, 50)
rsi_indicator(df, 14)
macd_indicator(df)

df.set_index('Date', inplace=True)

df[df.index.year == 2015].sample(5)

,Open,High,Low,Close,Volume,SMA_20,SMA_50,RSI,MACD,Bullish,Bearish
Date,,,,,,,,,,,
2015-01-01,2058.90,2072.36,1988.12,1994.99,7.733004e+10,2058.0510,2356.4964,46.364443,-87.294301,False,True
2015-05-01,2087.38,2134.72,2067.93,2107.39,6.518773e+10,2082.0095,2420.9196,45.956641,-114.610728,False,True
2015-10-01,1919.65,2094.32,1893.70,2079.36,8.584490e+10,2163.7435,2518.8310,35.521944,-155.215314,False,True
2015-02-01,1996.67,2119.59,1980.90,2104.50,6.877556e+10,2066.7150,2373.2846,52.491863,-87.321871,False,True
2015-11-01,2080.76,2116.48,2019.39,2080.41,7.594359e+10,2180.9460,2541.8594,33.173370,-166.076113,False,True


We can see that in these 5 rows, the stock in bearish in all of them meaning the share price is falling. This was calculated by checking whether the respective MACD was either positive or negaitve. 

I also found the total amount of days where the stock was oversold and overbought, which can be seen below

In [204]:
signals = rsi_signals("data/sp500_data.db", "sp500_table")


overbought_days = 0
oversold_days = 0
normal_days = 0

for date, rsi, label in signals:  # Noting which days the SP500 was oversold, overbought etc.
    if label == "overbought":
        overbought_days += 1
    elif label == "oversold":
        oversold_days += 1
    else:
        normal_days += 1

print(f"The total overbought days were: {overbought_days}"
      f"\nThe total oversold days were: {oversold_days}"
      f"\nThe total normal days were: {normal_days}"
      )

The total overbought days were: 22
The total oversold days were: 173
The total normal days were: 244


where normal days are where the product was neither oversold or overbought, in other words, if the rsi was between 30 and 70.

## Dashboard

To present my findings I've made a custom interactive dashboard, which can be found [here](https://sp500-analysis-kmhutxnyuhr5bljphbk8ym.streamlit.app/), through the use of the streamlit library in Python. It displays a quick preview of the data in a table, has a year slider so the user can choose their preferred year range they want to look at and 4 individual plots of:
- Open and Close prices
- Price
- RSI
- MACD


## Conclusion

This project demonstrates my skills in Python and SQL with libraries `pandas` and `streamlit`, as well as `mplfinance` for visualising my findings. It also showcases my understanding of financial data and how to communicate results clearly through the use of a dashboard.

This project could be added to in the future with a simple simulation of a trading strategy.

## Appendix 

See [requirements](requirements.txt) to see what was needed for this project.